In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

from src.stop_grid_mapper import StopGridMapper
from src.stop_explorer import StopExplorer
from src.grid_partitioning import Grid

### Main code

In [ ]:
# Create a GeoDataFrame for the stops dataset.
path_stops = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.stops.parquet'
stop_explorer = StopExplorer(path_stops)

display(stop_explorer.get_df_stops())
stop_explorer.get_df_stops().info()

### Materialize a uniform grid, with side of a given length, over the bounding box enclosing the stop segments.

In [ ]:
grid = Grid(grid_cell_length_meters = 50)
grid.compute_grid_over_geodata(stop_explorer.get_df_stops())
display(grid.get_grid())

# mappa = grid.generate_grid_map()
# mappa

### Compute the spatial join between the stop segments and the grid cells.

In [ ]:
# Instantiate the StopGridMapper class, which has the effect of computing the join between the centroids of the 
# stop segments and grid cells. This effectively maps each stop segment to the grid cell it falls into.
stop_grid_mapper = StopGridMapper(grid, stop_explorer)
display(stop_grid_mapper.get_join())

# Compute some aggregate statistics for each pair (cell_id, user_id).
stats_uid_cell = stop_grid_mapper.compute_statistics_cells_users()
display(stats_uid_cell)

stats_cell = stop_grid_mapper.compute_statistics_cells()
display(stats_cell)

In [ ]:
augmented_grid = grid.get_grid().copy().join(stats_cell, how='left').fillna(0)
display(augmented_grid)

In [ ]:
import folium
import branca.colormap as cm

gdf = augmented_grid.copy()
gdf["num_users"] = gdf["num_users"].fillna(0)

# Map center
minx, miny, maxx, maxy = gdf.total_bounds
m = folium.Map(location=[(miny + maxy) / 2, (minx + maxx) / 2],
               zoom_start=13, tiles="CartoDB positron")

# Colormap (robust to constant values)
vmin = float(gdf["num_users"].min())
vmax = float(gdf["num_users"].max())
if vmin == vmax:  # avoid degenerate scale
    vmin, vmax = 0.0, vmin if vmax > 0 else 1.0

cmap = cm.LinearColormap(
    colors=["#ffffcc","#a1dab4","#41b6c4","#2c7fb8","#253494"],
    vmin=vmin, vmax=vmax
).to_step(7)
cmap.caption = "Users per cell"
cmap.add_to(m)

# Polygon layer styled by num_users
folium.GeoJson(
    data=gdf.to_json(),
    style_function=lambda f: {
        "fillColor": cmap(f["properties"]["num_users"]),
        "color": "#666",
        "weight": 0.5,
        "fillOpacity": 0.7,
    },
    highlight_function=lambda f: {"weight": 2, "color": "black"},
    tooltip=folium.GeoJsonTooltip(
        fields=["num_users", "num_stops", "mean_duration_mins", "median_duration_mins"],
        aliases=["Users", "Stops", "Mean stop duration (min)", "Median stop duration (min)"],
        localize=True
    ),
    name="Grid (by users)"
).add_to(m)

folium.LayerControl().add_to(m)
m